In [1]:
# %pip install keras-tuner --upgrade
# %pip install tensorflow
# %pip install keras
# KerasTuner is a powerful and user-friendly hyperparameter tuning library built specifically for deep learning models in TensorFlow and Keras. It simplifies the often tedious and time-consuming process of finding the optimal configuration of hyperparameters for your model, leading to improved performance and efficiency.


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
#Loading the MNIST dataset in Keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
#This function returns a tuple of two NumPy arrays: one for the training data and one for the test data. Each array contains the images as well as the corresponding labels.

##print(x_train.shape) 3 dimensional array of 60000 elements containing the pixel values for the training data
##print(y_train.shape)  one dimensional array of 60000 elements containing the labels for the training data

#MNIST DATASET ---> 
#It consists of 60,000 28x28 pixel grayscale images of the 10 digits, along with a test set of 10,000 images. The MNIST dataset is widely used for training and evaluating machine learning models, particularly for image classification tasks.

In [4]:
print("Number of training examples :", x_train.shape[0], "and each image is of shape (%d, %d)"%(x_train.shape[1], x_train.shape[2]))
print("Number of training examples :", x_test.shape[0], "and each image is of shape (%d, %d)"%(x_test.shape[1], x_test.shape[2]))

Number of training examples : 60000 and each image is of shape (28, 28)
Number of training examples : 10000 and each image is of shape (28, 28)


In [5]:
# if you observe the input shape its 2 dimensional vector
# for each image we have a (28*28) vector
# we will convert the (28*28) vector into single dimensional vector of 1 * 784 using the reshape method of numpy arrays

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]) 
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]) 


In [6]:
# after converting the input images from 3d to 2d vectors

print("Number of training examples :", x_train.shape[0], "and each image is of shape (%d)"%(x_train.shape[1]))
print("Number of test examples :", x_test.shape[0], "and each image is of shape (%d)"%(x_test.shape[1]))

Number of training examples : 60000 and each image is of shape (784)
Number of test examples : 10000 and each image is of shape (784)


In [7]:
# if we observe the above matrix each cell is having a value between 0-255
# before we move to apply machine learning algorithms lets try to normalize the data
# X => (X - Xmin)/(Xmax-Xmin) = X/255

x_train = x_train/255
x_test = x_test/255

In [8]:
# Now we will convert the labels into one hot vectors using one-hot encoding
# EX: 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

print(y_train.shape)
print(y_train[0])

(60000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [9]:
# Now we will build the model architecture

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.models import Sequential
from keras_tuner import RandomSearch, Hyperband, BayesianOptimization

# start building a model
model = Sequential()

#First we will add the input layer
input_dim = x_train.shape[1]

model.add(Dense(units=input_dim, input_dim=input_dim, activation='relu'))

model.add(Dense(128, activation='sigmoid', input_shape=(input_dim,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='selu'))
model.add(Dropout(0.1))

## Adding the output layer
model.add(Dense(10, activation='softmax'))

# compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(x_train, y_train, epochs=5, batch_size=32, verbose = 1 , validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3021 - accuracy: 0.1298 - val_loss: 2.0212 - val_accuracy: 0.4520
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 1.5821 - accuracy: 0.4092 - val_loss: 0.8894 - val_accuracy: 0.7381
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.0325 - accuracy: 0.6355 - val_loss: 0.6100 - val_accuracy: 0.8242
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.7880 - accuracy: 0.7445 - val_loss: 0.4514 - val_accuracy: 0.8764
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.6559 - accuracy: 0.7975 - val_loss: 0.3817 - val_accuracy: 0.8913
Test accuracy: 0.8913000226020813


In [10]:
print(score)

[0.38172969222068787, 0.8913000226020813]


In [11]:
# Now we will build the model architecture

def build_model(optimizer):
    # start building a model
    model = Sequential()

    #First we will add the input layer
    input_dim = x_train.shape[1]

    model.add(Dense(units=input_dim, input_dim=input_dim, activation='relu'))

    model.add(Dense(128, activation='sigmoid', input_shape=(input_dim,)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='selu'))
    model.add(Dropout(0.1))

    ## Adding the output layer
    model.add(Dense(10, activation='softmax'))

    # compile the model
    if(optimizer == 'sgd'):
        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    elif(optimizer == 'adam'):
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    elif(optimizer == 'AdamW'):
        optimizer = tf.keras.optimizers.AdamW(learning_rate=0.1)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    elif(optimizer == 'Nadam'):
        optimizer = tf.keras.optimizers.Nadam(learning_rate=0.1)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

num_classes = 10

# A) Using SGD optimizer
print("Using SGD optimizer")
model = build_model('sgd')
accuracy = []
for i in range(num_classes):
    print("Class : ", i)
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose = 1 , validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy.append(score[1])
    print('Test accuracy:', score[1])

print("Average Accuracy : ", sum(accuracy)/num_classes)
accuracy.clear()

# B) Using Adam optimizer
print("Using Adam optimizer")
model = build_model('adam')
accuracy = []
for i in range(num_classes):
    print("Class : ", i)
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose = 1 , validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy.append(score[1])
    print('Test accuracy:', score[1])

print("Average Accuracy : ", sum(accuracy)/num_classes)
accuracy.clear()

# C) Using AdamW optimizer
print("Using Adam optimizer")
model = build_model('adamw')
accuracy = []
for i in range(num_classes):
    print("Class : ", i)
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose = 1 , validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy.append(score[1])
    print('Test accuracy:', score[1])

print("Average Accuracy : ", sum(accuracy)/num_classes)
accuracy.clear()

# B) Using Nadam optimizer
print("Using Nadam optimizer")
model = build_model('nadam')
accuracy = []
for i in range(num_classes):
    print("Class : ", i)
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose = 1 , validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy.append(score[1])
    print('Test accuracy:', score[1])

print("Average Accuracy : ", sum(accuracy)/num_classes)
accuracy.clear()

Using SGD optimizer
Class :  0
Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 2.2879 - accuracy: 0.1450 - val_loss: 1.9252 - val_accuracy: 0.3261
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6151 - accuracy: 0.4103 - val_loss: 0.9090 - val_accuracy: 0.7290
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.0445 - accuracy: 0.6443 - val_loss: 0.6284 - val_accuracy: 0.8154
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.8204 - accuracy: 0.7344 - val_loss: 0.4903 - val_accuracy: 0.8646
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.6915 - accuracy: 0.7858 - val_loss: 0.3964 - val_accuracy: 0.8944
Test accuracy: 0.8944000005722046
Class :  1
Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5955 - accuracy: 0.8194 - val_loss: 0.3385 - val_accuracy: 0.9075
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/ste

KeyboardInterrupt: 